In [ ]:
#default_exp redmine

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#export
import getpass
import re
import logging
from pathlib import Path
from typing import Iterable, Union
from collections import defaultdict
from redminelib import Redmine
import pandas as pd
from fastcore.test import *
from fastcore.basics import partialler, listify
from fastcore.script import Param, call_parse
from anateldb.constants import *
from anateldb.console import console

# Redmine

> Scripts para baixar informações sobre inspeções no Fiscaliza e atualizá-las.

## Dicionário de Dados
A seguir é mostrado um exemplo de dicionário de dados completo que é passado para ser validado, formatado e submetido à API do Redmine.
Os dados desse dicionário serão usados como teste das funções a seguir

In [ ]:
d = {}

d['Classe da Inspeção'] = 'Técnica'

d['Tipo de inspeção'] = 'Uso do Espectro - Monitoração'

d['Descrição da Inspeção'] = '''Atendimento da Denúncia AC202010213075425 (6104512), 
verificação da Potência e Intensidade de Campo Elétrico da Frequência 105.1MHz e seus harmônicos, 
além da checagem de Intermodulação e Espúrios nas frequências 450.3MHz e 750MHz.'''

d['Fiscal responsável '] = 'Ronaldo da Silva Alves Batista'

d['Fiscais '] = ['Ronaldo da Silva Alves Batista', 'Paulo Diogo Costa', 'Mario Augusto Volpini'] #string ou lista de strings

# A string sem o 'r' ( raw ) antes tende a ser interpretada incorretamente pelo Windows
d['Html'] = 'D:\\OneDrive - ANATEL\\Monitoramento\\Processos\\53504.0005432021-55\\Guarulhos.html'
           #r'D:\OneDrive - ANATEL\Monitoramento\Processos\53504.0005432021-55\Guarulhos.html'
           #'/d/OneDrive - ANATEL/Monitoramento/53504.0005432021-55/Guarulhos.html' 
            
d[' Gerar Relatório '] = 1 # int 0 ou 1

d['Frequência inicial']  = 54 #int ou float

d['Unidade da frequência inicial'] = 'MHz' #string

d['Frequência final'] = 700 #int ou float

d['Unidade da frequência final'] = 'MHz' #string

d['Data de início'] = '2021-03-19' #YYYY-MM-DD #string nesse formato

d['Data limite'] = '2021-12-31'  #YYYY-MM-DD #string nesse formato

d['UF/Município'] = "SP/São Paulo" #| ["SP/São Paulo", "SP/Sorocaba"] # String ou Lista de Strings

d['Serviços da Inspeção'] = ['230', '231', '800'] # String ou Lista de Strings

d['Qnt. de emissões na faixa'] = 12 # int

d['Emissões não autorizadas/desc'] = 70 # int

d['Horas de Preparação'] = 2 # int

d['Horas de Deslocamento'] = 0 # int

d['Horas de Execução'] = 32 # int

d['Horas de conclusão'] = 6 # int

d['Latitude (coordenadas)'] = -22.94694 # float

d['Longitude (coordenadas)'] = -43.21944 # float

d['Uso de PF'] = 'Não se aplica PF - uso apenas de formulários' # string

d['Ação de risco à vida criada?'] = 'Não' # string Sim | Não

d['Impossibilidade acesso online?'] = '0' # string '0' | '1'

d['notes'] = "Não foi constatada irregularidade no Período monitorado" # string

# No caso de uma tabela 

d['notes'] = """Faixa, Classe Especial, Classe A, Classe B, Classe C
                VHF-L,0,5,7,5
                VHF-H,0,12,1,0
                UHF,1,1,2,4
                FM,5,1,0,0
                RADCOM,0,0,0,0
                Outorgadas com indícios de irregularidades,1,2,3,4
            """

## Validação de Informações

In [ ]:
#export
def journal2table(journal):
    """Recebe a string journal, caso a formatação seja compatível com um csv, retorna este formato como markdown
    Do contrário simplesmente retorna a string inalterada"""
    table = [[r.strip() for r in j.strip().split(',')] for j in journal.split('\n') if j.strip() != '']
    if not len(set([len(t) for t in table])) == 1:
        print('A tabela não possui todas as linhas com o mesmo número de colunas')
        print('As notas serão salvas como texto somente')
        return table
    df = pd.DataFrame(table[1:], columns=table[0])
    return df.to_markdown(index=False, tablefmt='textile')

In [ ]:
def check_update(field: str, value , dtype, values_set: Iterable=None, val_text_string: bool = False):    
    if not isinstance(value, dtype):
        raise TypeError(f"É esperado que o campo {value} seja do tipo {str}, o fornecido foi {type(value)}")
    
    if values_set is not None and not set(listify(value)).issubset(set(values_set)):
        raise ValueError(f"O valor para {field} : {value} deve pertencer ao conjunto: {values_set}")
        
    if val_text_string:
        value = value_text_string(value)
        
    return {'id': FIELD2ID[field], 'value': value}
        
def validate_datadict(data_dict: dict, insp: str, fiscaliza: Redmine)->dict:
    
    assert isinstance(data_dict, dict), f"É esperado um dicionário como entrada, {data_dict} é do tipo: {type(data_dict)}" 
    
    if not set(data_dict.keys()).issubset(DICT_FIELDS):
        raise ValueError(f'As chaves seguintes são desconhecidas ou estão com o nome diferente do esperado: \
                         {set(data_dict.keys()).difference(DICT_FIELDS)}')
        
    assert isinstance(fiscaliza, Redmine), f"Uma instância do tipo Redmine é esperada, foi retornado objeto fiscaliza {type(fiscaliza)}"
        
    valida_fiscaliza(fiscaliza)    
    issue = fiscaliza.issue.get(insp, include=['relations', 'attachments'])
    issue_id = issue.id    
    date_pattern = '([2]\d{3})-(0[1-9]|1[0-2])-(0[1-9]|[12]\d|3[01])'    
    d = data_dict.copy()
    
    key = DICT_FIELDS[0]
    if (classe := d.get(key)):
        d[key] = check_update(key, classe, str, CLASSE, True)
        
    key = DICT_FIELDS[1]
    if (tipo := d.get(key)):
        d[key] = check_update(key, tipo, str, TIPO, True)
        
    key = DICT_FIELDS[2]
    if (description := d.get(key)):
        d[key] = check_update(key, description, str)
    
    key = DICT_FIELDS[3]
    if (fiscal := d.get(key)):
        _, name2id = issue2users(fiscaliza, issue_id)
        d[key] = check_update(key, fiscal, str, name2id.keys())
        
    key = DICT_FIELDS[4]
    if (fiscais := d.get(key)):
        _, name2id = issue2users(fiscaliza, issue_id)
        fiscais = listify(fiscais)
        d[key] = check_update(key, fiscais, list, name2id.keys())
        
    key = DICT_FIELDS[5]
    if (html := d.get(key, None)) is not None:
        html = Path(html)
        assert html.exists() and html.is_file(), f"O caminho para o arquivo html informado não existe ou não é um arquivo: {html}"
        d[key] = check_update(key, html.read_text(), str)
        key = DICT_FIELDS[6]
        if (relatorio := d.get(key, None)) is not None:
            d[key] = check_update(key, relatorio, int, (0,1))
    
    key = DICT_FIELDS[7]
    if (freq_init := d.get(key)):
        d[key] = check_update(key, freq_init, (int,float))
    
    key = DICT_FIELDS[8]
    if (init_unit := d.get(key)):
        d[key] = check_update(key, init_unit, str, ('kHz', 'MHz', 'GHz'))
    
    key = DICT_FIELDS[9]
    if (freq_final := d.get(key)):
        d[key] = check_update(key, freq_final, (int,float))
    
    key = DICT_FIELDS[10]
    if (final_unit := d.get(key)):
        d[key] = check_update(key, final_unit, str, ('kHz', 'MHz', 'GHz'))
        
    key = DICT_FIELDS[11]
    if (start_date := d.get(key)):
        assert re.match(date_pattern, start_date), f'A data informada é inválida {start_date}, informe o formato yyyy-mm-dd'
        d[key] = start_date
    else:
        raise ValueError(f'O campo "Data de início" não pode ficar vazio!')
    
    key = DICT_FIELDS[12]
    if (due_date := d.get(key)):
        assert re.match(date_pattern, due_date), f'A data informada é inválida {due_date}, informe o formato yyyy-mm-dd'
        d[key] = due_date
    else:
        raise ValueError(f'O campo "Data limite" não poder ficar vazio!')        
    
    key = DICT_FIELDS[13]
    if (municipio := d.get(key)):
        municipios = pd.read_pickle('municipios.pkl')
        municipio = listify(municipio)
        lista_municipios = []
        for m in municipio:
            match =  re.match(f'({"|".join(ESTADOS)})/(\w+[\s|\w]+)', m)
            if not match:
                raise ValueError(f'Verifique o formato da string UF/Município: {m}')                    
            lista_municipios.append(check_update(key, m, str, municipios, True)['value'])        
        d[key] = {'id': FIELD2ID[key], 'value': lista_municipios}
        del municipios
        
    key = DICT_FIELDS[14]
    if (servicos := d.get(key)):
        servicos = listify(servicos)
        lista_servicos = []
        for s in servicos:
            s = SERVICOS[s]
            lista_servicos.append(check_update(key, s, str, SERVICOS.values(), True)['value'])
        d[key] = {'id': FIELD2ID[key], 'value': lista_servicos}
        
    key = DICT_FIELDS[15]
    if (qnt := d.get(key)) is not None: # 0 não deve ser interpretado como False
        d[key] = check_update(key, qnt, int)
    
    key = DICT_FIELDS[16]
    if (nauto := d.get(key)) is not None: 
        d[key] = check_update(key, nauto, int)
        
    key = DICT_FIELDS[17]
    if (hprep := d.get(key)) is not None:
        d[key] = check_update(key, hprep, int)
        
    key = DICT_FIELDS[18]
    if (hdesl := d.get(key)) is not None:
        d[key] = check_update(key, hdesl, int)
        
    key = DICT_FIELDS[19]
    if (hexec := d.get(key)) is not None:
        d[key] = check_update(key, hexec, int)
        
    key = DICT_FIELDS[20]
    if (hconc := d.get(key)) is not None:
        d[key] = check_update(key, hconc, int)
     
    key = DICT_FIELDS[21]
    if (lat := d.get(key)) is not None:
        max_lat = 5.2666664 # Monte Caburaí RR
        min_lat = -33.7017531 # Arroio Chuy RS
        if not min_lat <= lat <= max_lat:
            raise ValueError(f'O valor de latitude inserido está fora dos extremos do Brasil: ({min_lat}, {max_lat})')
        d[key] = check_update(key, lat, float)
        
    key = DICT_FIELDS[22]
    if (long := d.get(key)): # Não pode ser 0
        min_long = -75.3709938
        max_long = -32.423786
        if not min_long <= long <= max_long:
            raise ValueError(f'O valor de longitude inserido está fora dos extremos do Brasil: ({min_long}, {max_long})')
        d[key] = check_update(key, long, float)
    
    key = DICT_FIELDS[23]
    if (pf := d.get(key)):
        d[key] = check_update(key, pf, str, PF)
        
    key = DICT_FIELDS[24]
    if (risco := d.get(key)):
        d[key] = check_update(key, risco, str, ('Sim', 'Não'))
        
    key = DICT_FIELDS[25]
    if (online := d.get(key)):
        d[key] = check_update(key, online, str, ('0', '1'))
        
    if (notes := d.get('notes')):
        d['notes'] = journal2table(notes)
            
    return d

## Funções Auxiliares / Informações de Inspeção / Ação / Autenticação

In [ ]:
#export
def value_text_string(input_value):
    return '{' + '"valor":"{0}","texto":"{0}"'.format(input_value) + '}'

def auth_user(username, password, teste=True, verify=True):
    url = URLHM if teste else URL
    fiscaliza = Redmine(url, username=username, password=password, requests={'verify': verify})
    fiscaliza.auth()
    return fiscaliza

def valida_fiscaliza(fiscaliza_obj: Redmine)->None:
    if not isinstance(fiscaliza_obj, Redmine):
        raise TypeError(f"O Objeto Fiscaliza deve ser uma instância autenticada "\
        "(logada) da classe Redmine, o typo do objeto fornecido é {type(fiscaliza_obj)}")

def issue_type(fiscaliza, insp):
    if (tipo := fiscaliza.issue.get(insp).tracker['id']) == 1:
        return 'Inspeção'
    elif tipo == 2:
        return 'Ação'
    return 'Desconhecido'
    

def issue2users(fiscaliza: Redmine, issue: str)-> dict:
    """Recebe objeto Redmine e string issue e retorna um dicionário com os usuários do grupo Inspeção-Execução"""
    valida_fiscaliza(fiscaliza)
    proj = fiscaliza.issue.get(issue).project.name.lower()
    members = fiscaliza.project_membership.filter(project_id=proj)
    id2name = {}
    name2id = {}
    names = []
    for member in members:
        if roles := getattr(member, 'roles', []):
            for role in roles:
                if str(role) == 'Inspeção-Execução':
                    if user := getattr(member, 'user', None):
                        if (id_ := getattr(user, 'id', None)) and (name := getattr(user, 'name', None)):
                            names.append((id_, name))
                            
    names.sort(key=lambda x: x[1])
    id2name = dict(names)
    name2id = {v:k for k,v in id2name.items()}
    return id2name, name2id

def insp2acao(insp: str, fiscaliza: Redmine)->dict:        
    """Recebe o objeto `fiscaliza` e a string referente à inspeção `insp` e retorna um dicionário resumo da Ação atrelada à inspeção

    Args:
        redmineObj (Redmine): Objeto Redmine autenticado
        insp (str): string com o número da inspeção

    Returns:
        dict: Dicionário com o id, nome e descrição da Ação associada à inspeção
    >>>fiscaliza = Redmine(URL, username=USR, password=PWD)
       fiscaliza.auth()
       detalhar_inspecao(fiscaliza, '51804')
    {'id': 51803,
    'name': 'ACAO_GR01_2021_0456',
    'description': 'Atendimento à Denúncia AC202010213075425 (6104512)'}
    """
    valida_fiscaliza(fiscaliza)
    issue = fiscaliza.issue.get(insp, include=['relations', 'attachments'])
    if relations := getattr(issue, 'relations', []):
        if relations := getattr(relations, 'values', []):
            relations = relations()
    for relation in relations:
        if (issue_to_id := relation.get('issue_to_id', None)):
            if (issue_to_id := fiscaliza.issue.get(issue_to_id)):
                if 'ACAO' in str(issue_to_id) or ((tracker := getattr(issue_to_id, 'tracker', None)) and (getattr(tracker, 'id', None) == 2)):
                    if (description := getattr(issue_to_id, 'custom_fields', None)) is not None:
                        if description := description.get(ACAO_DESCRIPTION, None):
                            description = getattr(description, 'value', '')
                        else:
                            description = ''
                    else:
                        description = ''
                    return {'id': getattr(issue_to_id, 'id', ''), 'nome': str(issue_to_id), 'descrição': description}
    else:
        return {'id': '', 'nome': '', 'descrição': ''}

def detalhar_inspecao(insp: str, fiscaliza: Redmine=None)->dict:
    """Recebe número da inspeção `insp` e objeto Redmine logado `fiscaliza`
    Retorna um dicionário com a Situação e campos preenchidos da Inspeção"""
    valida_fiscaliza(fiscaliza)
    result = {k:'' for k in FIELDS}
    issue = fiscaliza.issue.get(insp, include=['relations', 'attachments'])
    result.update({k: str(getattr(issue,k, '')) for k in FIELDS})
    if custom_fields := getattr(issue, 'custom_fields', None):
        custom_fields = list(custom_fields)
        for field in custom_fields:
            key = field.id
            if key in CUSTOM_IDS:
                if (value := getattr(field, 'value')):
                    result[ID2FIELD[key]] = value
    return result

In [ ]:
#export
def atualiza_fiscaliza(insp, fields, fiscaliza, status, notes=None):
    """ Atualiza a Inspeção para a Situação `status` com os dados do dicionário `fields`"""
    assert status in STATUS, f'Digite uma das mudanças de situação válidas: {STATUS.keys()}'
    valida_fiscaliza(fiscaliza)    
    issue = fiscaliza.issue.get(insp, include=['relations', 'attachments'])
    issue_status = str(getattr(issue, 'status', ''))
    if issue_status == status:
        logging.info(f"A inspeção atual já está no status desejado: {status}.")
    custom_fields = [fields.get(field, '') for field in STATUS[status]]
    if status in ('Relatando', 'Relatada'):
        start_date = fields.get('Data de início', '')
        due_date = fields.get('Data limite', '')
    else:
        start_date, due_date= None, None    
    notes = fields.get('notes') if status == 'Relatada' else None
    return fiscaliza.issue.update(issue.id, status_id=SITUACAO[status], custom_fields=custom_fields, start_date=start_date, due_date=due_date, notes=notes)

In [ ]:
#export
@call_parse
def relatar_inspecao(login: Param("Login Anatel do Usuário", str),
                     senha: Param("Senha Utilizada nos Sistemas Interativos da Anatel", str), 
                     inspecao: Param("Número da Inspeção a ser relatada", str),
                     dados: Param("Dicionário com os Dados a serem relatados", dict)):
    """Relata a inspeção `inspecao` com os dados constantes no dicionário `dados`"""
    fiscaliza = auth_user(login, senha)
    console.print("Usuário Autenticado com Sucesso :thumbs_up:", style="bold green")
    
    if (tipo := issue_type(fiscaliza, inspecao)) == 'Ação':
        console.print(f':exclamation: O número de inspeção inserido {inspecao} corresponde a uma [bold red]Ação[/bold red] :exclamation:')
        return 
    
    acao = insp2acao(inspecao, fiscaliza)
    console.print(f"Inspeção {inspecao} vinculada à Ação {acao}")
    
    with console.status("[magenta]Validando o dicionário de dados...", spinner='monkey') as status:
        data_dict = validate_datadict(d, inspecao, fiscaliza)
        console.print("[bold green] Dados Validados com Sucesso :raised_hands:")
        
    console.rule('Situação atual da Inspeção')
    status_atual = detalhar_inspecao(inspecao, fiscaliza) 
    console.print(status_atual)
    antes = status_atual['status']
    lista_status = list(SITUACAO.keys())
    
    index = lista_status.index(antes)+1
    if index >= len(lista_status):
        index = len(lista_status) - 1
    
    for status in lista_status[index:]:
        with console.status(f'Atualizando [bold_yellow]{antes}[/bold_yellow] para [bold_green]{status}', spinner='arrow3'):
            atualiza_fiscaliza(inspecao, data_dict, fiscaliza, status)
            console.rule('Sucesso :sparkles:')
        detalhes = detalhar_inspecao(inspecao, fiscaliza)
        console.print(detalhes)
        
        if antes == 'Em andamento' and status == 'Relatando':
            console.rule('Atenção :exclamation:')
            console.print(f"Assine o Relatório de Monitoramento: {detalhes['Relatório de Monitoramento']} e chame a função novamente :exclamation:")
            break
        antes = status
        
    if detalhes['status'] == 'Relatada':
        console.print('Inspeção Relatada :sunglasses:')

In [ ]:
USR = getpass.getuser()
PWD = getpass.getpass()

 ··········


In [ ]:
relatar_inspecao(USR, PWD, '52876', d)

Usuário Autenticado com Sucesso 👍

❗ O número de inspeção inserido 52876 corresponde a uma Ação ❗

## Testes

In [ ]:
d = defaultdict(str)
f = partialler(validate_datadict, d)
test_eq(f(), True)

In [ ]:
d = [('Servidor', 'Ronaldo')]
with ExceptionExpected(ex=AssertionError, regex='É esperado um dicionário'):
    validate_datadict(d)

In [ ]:
d = {}
d['Servidor'] = 'Ronaldo'
with ExceptionExpected(ex=ValueError, regex='As chaves seguintes são desconhecidas'):
    validate_datadict(d)

In [ ]:
with ExceptionExpected(ex=ValueError):
    validate_datadict({'Classe da Inspeção': 'Outorga'})

In [ ]:
with ExceptionExpected(ex=TypeError):
    validate_datadict({'Classe da Inspeção': ['Técnica']})
    validate_datadict({'Classe da Inspeção': 12})

### Mostrar resumo da Ação à partir da Inspeção

In [ ]:
#hide
insp2acao(INSP, fiscaliza)

In [ ]:
detalhar_inspecao(INSP, fiscaliza)

In [ ]:
from nbdev.export import notebook2script
notebook2script()